In [1]:
import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

from scipy.spatial.transform import Rotation as R

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

import polars as pl

import matplotlib.pyplot as plt

In [7]:
def remove_gravity_from_acc(acc_data:pd.DataFrame, rot_data:pd.DataFrame):
    acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.81])
    for i in range(len(acc_values)):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :]
            continue
        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
    return linear_accel

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200):
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_vel = np.zeros((len(quat_values), 3))
    for i in range(len(quat_values) - 1):
        q_t, q_t_plus_dt = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q_t)) or np.all(np.isnan(q_t_plus_dt)): continue
        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)
            delta_rot = rot_t.inv() * rot_t_plus_dt
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError: pass
    return angular_vel

def calculate_angular_distance(rot_data):
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_dist = np.zeros(len(quat_values))
    for i in range(len(quat_values) - 1):
        q1, q2 = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q1)) or np.all(np.isnan(q2)): continue
        try:
            r1, r2 = R.from_quat(q1), R.from_quat(q2)
            relative_rotation = r1.inv() * r2
            angular_dist[i] = np.linalg.norm(relative_rotation.as_rotvec())
        except ValueError: pass
    return angular_dist

In [4]:
!pwd

Now using node v22.16.0 (npm v11.4.1)
/home/karunru/Home/Kaggle/kaggle_monorepo/projects/CMI_Detect_Behavior_with_Sensor_Data/outputs/claude/tmp


In [5]:
RAW_DIR = Path("../../../data")

In [6]:
df = pd.read_csv(RAW_DIR / "train.csv")
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,62.0,65.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,71.0
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,71.0,72.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,80.0,77.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,72.0,77.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [9]:
group_df = df.loc[df["sequence_id"]=="SEQ_000007"]
group_df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,SEQ_000007_000005,Target,SEQ_000007,5,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,3.996094,...,-1.0,-1.0,118.0,114.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,SEQ_000007_000006,Target,SEQ_000007,6,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,4.035156,...,-1.0,-1.0,104.0,118.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,SEQ_000007_000007,Target,SEQ_000007,7,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,3.730469,...,-1.0,-1.0,105.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,SEQ_000007_000008,Target,SEQ_000007,8,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,4.535156,...,-1.0,-1.0,103.0,122.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,SEQ_000007_000009,Target,SEQ_000007,9,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,3.921875,...,-1.0,-1.0,104.0,123.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [10]:
pd.DataFrame(remove_gravity_from_acc(group_df[['acc_x', 'acc_y', 'acc_z']], group_df[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group_df.index)

,linear_acc_x,linear_acc_y,linear_acc_z
0,-0.138540,0.044578,-0.053696
1,0.237503,0.238219,-0.808055
2,-0.469262,0.526305,-0.412869
3,0.619349,0.933462,-0.871046
4,1.226582,0.567647,0.839761
5,0.241013,0.785693,0.764923
6,0.135716,-0.277350,0.422413
7,-0.297481,-0.095396,0.329970
8,0.425583,-0.147223,0.131000
9,-0.132633,0.160297,0.305051


In [11]:
def remove_gravity_from_acc_pl(df:pl.DataFrame, tol: float = 1e-6) -> pl.DataFrame:
    # 必要列がある前提で横結合（行順を対応させる）
    df = df.clone()

    x = pl.col("rot_x")
    y = pl.col("rot_y")
    z = pl.col("rot_z")
    w = pl.col("rot_w")

    # 有効判定：NaNなし & ノルムがしきい値超え
    norm = (x**2 + y**2 + z**2 + w**2).sqrt()
    valid = (~pl.any_horizontal([x.is_null(), y.is_null(), z.is_null(), w.is_null()])) & (norm > tol)

    # 正規化四元数
    xn = x / norm
    yn = y / norm
    zn = z / norm
    wn = w / norm

    # 逆回転後の重力（センサ座標）
    gx = 19.62 * (xn * zn - wn * yn)
    gy = 19.62 * (wn * xn + yn * zn)
    gz = 9.81  - 19.62 * (xn * xn + yn * yn)

    # 無効行は重力0扱いにして acc をそのまま通す
    gx = pl.when(valid).then(gx).otherwise(0.0)
    gy = pl.when(valid).then(gy).otherwise(0.0)
    gz = pl.when(valid).then(gz).otherwise(0.0)

    # 線形加速度 = 測定 - 重力（有効行） / 無効行は測定のまま
    lin_x = pl.when(valid).then(pl.col("acc_x") - gx).otherwise(pl.col("acc_x")).alias("linear_acc_x")
    lin_y = pl.when(valid).then(pl.col("acc_y") - gy).otherwise(pl.col("acc_y")).alias("linear_acc_y")
    lin_z = pl.when(valid).then(pl.col("acc_z") - gz).otherwise(pl.col("acc_z")).alias("linear_acc_z")

    return df.with_columns([lin_x, lin_y, lin_z]).select("linear_acc_x", "linear_acc_y", "linear_acc_z")

In [14]:
(remove_gravity_from_acc_pl(pl.DataFrame._from_pandas(group_df)) - pl.from_pandas(pd.DataFrame(remove_gravity_from_acc(group_df[['acc_x', 'acc_y', 'acc_z']], group_df[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group_df.index))).sum()


linear_acc_x,linear_acc_y,linear_acc_z
f64,f64,f64
0.0,0.0,2.2204e-15


In [15]:
pd.DataFrame(calculate_angular_velocity_from_quat(group_df[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group_df.index)

,angular_vel_x,angular_vel_y,angular_vel_z
0,-6.600812,9.554974,2.140512
1,-16.678839,44.579519,21.057952
2,-34.657626,27.425885,37.117944
3,-55.731838,43.651922,23.575392
4,-18.596728,11.683623,15.289419
5,-3.013677,-4.939589,9.500976
6,-0.010911,-2.820534,-0.501202
7,-0.904489,-1.356542,-2.574271
8,-1.246530,1.381962,-0.618971
9,-0.293860,0.128209,-0.608901


In [16]:
def calculate_angular_velocity_from_quat_pl(rot_pl: pl.DataFrame, time_delta: float = 1/200, tol: float = 1e-8) -> pl.DataFrame:
    # rot_pl: columns = rot_x, rot_y, rot_z, rot_w（スカラー末尾）
    x1 = pl.col("rot_x"); y1 = pl.col("rot_y"); z1 = pl.col("rot_z"); w1 = pl.col("rot_w")
    x2 = x1.shift(-1);      y2 = y1.shift(-1);      z2 = z1.shift(-1);      w2 = w1.shift(-1)

    # ノルムで正規化（SciPy 相当）
    n1 = (x1*x1 + y1*y1 + z1*z1 + w1*w1).sqrt()
    n2 = (x2*x2 + y2*y2 + z2*z2 + w2*w2).sqrt()
    xn1, yn1, zn1, wn1 = x1/n1, y1/n1, z1/n1, w1/n1
    xn2, yn2, zn2, wn2 = x2/n2, y2/n2, z2/n2, w2/n2

    # 連続性のための符号合わせ（dot<0なら後者を反転）
    dot = xn1*xn2 + yn1*yn2 + zn1*zn2 + wn1*wn2
    sgn = pl.when(dot.is_not_null() & (dot < 0)).then(-1.0).otherwise(1.0)
    xn2, yn2, zn2, wn2 = xn2*sgn, yn2*sgn, zn2*sgn, wn2*sgn

    # 単位四元数を前提に delta = q1^{-1} * q2 = conj(q1) * q2 を計算
    # conj(q1)=(-x1,-y1,-z1,w1)。四元数積の展開（スカラー末尾）
    dx = wn1*xn2 - xn1*wn2 - yn1*zn2 + zn1*yn2
    dy = wn1*yn2 + xn1*zn2 - yn1*wn2 - zn1*xn2
    dz = wn1*zn2 - xn1*yn2 + yn1*xn2 - zn1*wn2
    dw = wn1*wn2 + xn1*xn2 + yn1*yn2 + zn1*zn2

    # 数値安全のためクランプ
    dw = pl.min_horizontal(pl.lit(1.0), pl.max_horizontal(pl.lit(-1.0), dw))

    # 回転ベクトル = 角度 * 単位軸
    vnorm = (dx*dx + dy*dy + dz*dz).sqrt()
    angle = 2.0 * pl.arctan2(vnorm, dw)          # より安定（acos より端がマシ）
    scale = pl.when(vnorm > tol).then(angle / vnorm).otherwise(0.0)

    rvx = dx * scale
    rvy = dy * scale
    rvz = dz * scale

    # 角速度 [rad/s]
    wx = rvx / time_delta
    wy = rvy / time_delta
    wz = rvz / time_delta

    # 有効判定（NaN/Nullや最終行は無効 → 0）
    valid = (
        (~pl.any_horizontal([x1.is_null(), y1.is_null(), z1.is_null(), w1.is_null()])) &
        (~pl.any_horizontal([x2.is_null(), y2.is_null(), z2.is_null(), w2.is_null()])) &
        (n1 > tol) & (n2 > tol)
    )

    return rot_pl.with_columns([
        pl.when(valid).then(wx).otherwise(0.0).alias("ang_vel_x"),
        pl.when(valid).then(wy).otherwise(0.0).alias("ang_vel_y"),
        pl.when(valid).then(wz).otherwise(0.0).alias("ang_vel_z"),
    ]).select("ang_vel_x", "ang_vel_y", "ang_vel_z")

In [28]:
(calculate_angular_velocity_from_quat_pl(pl.from_pandas(group_df)) - pl.from_pandas(pd.DataFrame(calculate_angular_velocity_from_quat(group_df[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group_df.index)
)).sum()

ang_vel_x,ang_vel_y,ang_vel_z
f64,f64,f64
8.4219e-14,-1.0714e-14,-1.7739e-13


In [29]:
pd.DataFrame(calculate_angular_distance(group_df[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_distance'], index=group_df.index)

,angular_distance
0,0.059044
1,0.260238
2,0.288577
3,0.373073
4,0.133801
5,0.055622
6,0.014324
7,0.015236
8,0.009807
9,0.003441


In [31]:
def calculate_angular_distance_pl(rot_pl: pl.DataFrame, tol: float = 1e-8) -> pl.DataFrame:
    # 入力: rot_x, rot_y, rot_z, rot_w（スカラー末尾）
    x1 = pl.col("rot_x"); y1 = pl.col("rot_y"); z1 = pl.col("rot_z"); w1 = pl.col("rot_w")
    x2 = x1.shift(-1);      y2 = y1.shift(-1);      z2 = z1.shift(-1);      w2 = w1.shift(-1)

    # 正規化（SciPy相当）
    n1 = (x1*x1 + y1*y1 + z1*z1 + w1*w1).sqrt()
    n2 = (x2*x2 + y2*y2 + z2*z2 + w2*w2).sqrt()
    xn1, yn1, zn1, wn1 = x1/n1, y1/n1, z1/n1, w1/n1
    xn2, yn2, zn2, wn2 = x2/n2, y2/n2, z2/n2, w2/n2

    # q と -q の同値対策（最短経路）
    dot = xn1*xn2 + yn1*yn2 + zn1*zn2 + wn1*wn2
    sgn = pl.when(dot.is_not_null() & (dot < 0)).then(-1.0).otherwise(1.0)
    xn2, yn2, zn2, wn2 = xn2*sgn, yn2*sgn, zn2*sgn, wn2*sgn

    # delta = q1^{-1} * q2 = conj(q1) * q2 （スカラー末尾の四元数積）
    dx = wn1*xn2 - xn1*wn2 - yn1*zn2 + zn1*yn2
    dy = wn1*yn2 + xn1*zn2 - yn1*wn2 - zn1*xn2
    dz = wn1*zn2 - xn1*yn2 + yn1*xn2 - zn1*wn2
    dw = wn1*wn2 + xn1*xn2 + yn1*yn2 + zn1*zn2

    # 数値安定化＆角度（0..π）
    vnorm = (dx*dx + dy*dy + dz*dz).sqrt()
    angle = 2.0 * pl.arctan2(vnorm, dw)  # = ||as_rotvec||

    # 有効判定（末尾行やNaN/ゼロ長は 0）
    valid = (
        (~pl.any_horizontal([x1.is_null(), y1.is_null(), z1.is_null(), w1.is_null()])) &
        (~pl.any_horizontal([x2.is_null(), y2.is_null(), z2.is_null(), w2.is_null()])) &
        (n1 > tol) & (n2 > tol)
    )

    return rot_pl.with_columns(
        pl.when(valid).then(angle).otherwise(0.0).alias("angular_dist")
    ).select("angular_dist")

In [34]:
(calculate_angular_distance_pl(pl.from_pandas(group_df)) - pl.from_pandas(pd.DataFrame(calculate_angular_distance(group_df[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_distance'], index=group_df.index))).sum()

angular_dist
f64
1.3314e-16


In [ ]:
print("▶ TRAIN MODE – loading dataset …")
train_dem_df = pd.read_csv(RAW_DIR / "train_demographics.csv")
df = pd.merge(df, train_dem_df, on='subject', how='left')
le = LabelEncoder()
df['gesture_int'] = le.fit_transform(df['gesture'])
np.save(EXPORT_DIR / "gesture_classes.npy", le.classes_)

# --- [変更点] 物理FEの導入 ---
print("  Removing gravity and calculating linear acceleration features...")
linear_accel_list = [pd.DataFrame(remove_gravity_from_acc(group[['acc_x', 'acc_y', 'acc_z']], group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group.index) for _, group in df.groupby('sequence_id')]
df = pd.concat([df, pd.concat(linear_accel_list)], axis=1)
df['linear_acc_mag'] = np.sqrt(df['linear_acc_x']**2 + df['linear_acc_y']**2 + df['linear_acc_z']**2)
df['linear_acc_mag_jerk'] = df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0)

print("  Calculating angular velocity and distance from quaternions...")
angular_vel_list = [pd.DataFrame(calculate_angular_velocity_from_quat(group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group.index) for _, group in df.groupby('sequence_id')]
df = pd.concat([df, pd.concat(angular_vel_list)], axis=1)
angular_dist_list = [pd.DataFrame(calculate_angular_distance(group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_distance'], index=group.index) for _, group in df.groupby('sequence_id')]
df = pd.concat([df, pd.concat(angular_dist_list)], axis=1)

# --- [変更点] 物理FEを反映した特徴量リスト ---
imu_cols_base = ['linear_acc_x', 'linear_acc_y', 'linear_acc_z'] + [c for c in df.columns if c.startswith('rot_')]
imu_engineered = ['linear_acc_mag', 'linear_acc_mag_jerk', 'angular_vel_x', 'angular_vel_y', 'angular_vel_z', 'angular_distance']
imu_cols = list(dict.fromkeys(imu_cols_base + imu_engineered))

thm_cols_original = [c for c in df.columns if c.startswith('thm_')]
tof_aggregated_cols_template = []
for i in range(1, 6): tof_aggregated_cols_template.extend([f'tof_{i}_mean', f'tof_{i}_std', f'tof_{i}_min', f'tof_{i}_max'])

final_feature_cols = imu_cols + thm_cols_original + tof_aggregated_cols_template
imu_dim_final = len(imu_cols)
tof_thm_aggregated_dim_final = len(thm_cols_original) + len(tof_aggregated_cols_template)

print(f"  IMU (phys-based) {imu_dim_final} | THM + Aggregated TOF {tof_thm_aggregated_dim_final} | total {len(final_feature_cols)} features")
np.save(EXPORT_DIR / "feature_cols.npy", np.array(final_feature_cols))

print("  Building sequences...")
seq_gp = df.groupby('sequence_id')
X_list_unscaled, y_list_int, groups_list, lens = [], [], [], []
for seq_id, seq_df in seq_gp:
    seq_df_copy = seq_df.copy()
    for i in range(1, 6):
        pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]; tof_data = seq_df_copy[pixel_cols].replace(-1, np.nan)
        seq_df_copy[f'tof_{i}_mean'], seq_df_copy[f'tof_{i}_std'], seq_df_copy[f'tof_{i}_min'], seq_df_copy[f'tof_{i}_max'] = tof_data.mean(axis=1), tof_data.std(axis=1), tof_data.min(axis=1), tof_data.max(axis=1)
    X_list_unscaled.append(seq_df_copy[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32'))
    y_list_int.append(seq_df_copy['gesture_int'].iloc[0])
    groups_list.append(seq_df_copy['subject'].iloc[0])
    lens.append(len(seq_df_copy))

print("  Fitting StandardScaler...")
all_steps_concatenated = np.concatenate(X_list_unscaled, axis=0)
scaler = StandardScaler().fit(all_steps_concatenated)
joblib.dump(scaler, EXPORT_DIR / "scaler.pkl")

print("  Scaling and padding sequences...")
X_scaled_list = [scaler.transform(x_seq) for x_seq in X_list_unscaled]
pad_len = int(np.percentile(lens, PAD_PERCENTILE)); np.save(EXPORT_DIR / "sequence_maxlen.npy", pad_len)
X = pad_sequences(X_scaled_list, maxlen=pad_len, padding='post', truncating='post', dtype='float32')
y_stratify, groups, y = np.array(y_list_int), np.array(groups_list), to_categorical(y_list_int, num_classes=len(le.classes_))

print("  Starting training with Stratified Group K-Fold CV...")
sgkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
oof_preds = np.zeros_like(y, dtype='float32')

for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y_stratify, groups)):
    print(f"\n===== FOLD {fold+1}/{N_SPLITS} =====")
    X_tr, X_val, y_tr, y_val = X[train_idx], X[val_idx], y[train_idx], y[val_idx]
    model = build_gated_two_branch_model(pad_len, imu_dim_final, tof_thm_aggregated_dim_final, len(le.classes_), wd=WD)
    model.compile(optimizer=Adam(LR_INIT),
                  loss={'main_output': tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), 'tof_gate': tf.keras.losses.BinaryCrossentropy()},
                  loss_weights={'main_output': 1.0, 'tof_gate': GATE_LOSS_WEIGHT},
                  metrics={'main_output': 'accuracy'})
    class_weight_dict = dict(enumerate(compute_class_weight('balanced', classes=np.arange(len(le.classes_)), y=y_tr.argmax(1))))
    train_gen = GatedMixupGenerator(X_tr, y_tr, batch_size=BATCH_SIZE, imu_dim=imu_dim_final, class_weight=class_weight_dict, alpha=MIXUP_ALPHA, masking_prob=MASKING_PROB)
    val_gen = GatedMixupGenerator(X_val, y_val, batch_size=BATCH_SIZE, imu_dim=imu_dim_final)
    cb = EarlyStopping(patience=PATIENCE, restore_best_weights=True, verbose=1, monitor='val_main_output_accuracy', mode='max')
    model.fit(train_gen, epochs=EPOCHS, validation_data=val_gen, callbacks=[cb], verbose=1)
    model.save(EXPORT_DIR / f"gesture_model_fold_{fold}.h5")
    preds_val, _ = model.predict(X_val); oof_preds[val_idx] = preds_val

# (OOFスコア計算部分はcmi_2025_metric_copy_for_import.pyがあれば有効化)
print("\n✔ Training done.")

# 評価指標ファイルをインポートしてOOFスコアを計算
from cmi_2025_metric_copy_for_import import CompetitionMetric
true_oof_int = y.argmax(1)
pred_oof_int = oof_preds.argmax(1)

h_f1_oof = CompetitionMetric().calculate_hierarchical_f1(
    pd.DataFrame({'gesture': le.classes_[true_oof_int]}),
    pd.DataFrame({'gesture': le.classes_[pred_oof_int]}))
print(f"Overall OOF H‑F1 Score = {h_f1_oof:.4f}")

else:
print("▶ INFERENCE MODE – loading artefacts from", PRETRAINED_DIR)
final_feature_cols = np.load(PRETRAINED_DIR / "feature_cols.npy", allow_pickle=True).tolist()
pad_len        = int(np.load(PRETRAINED_DIR / "sequence_maxlen.npy"))
scaler         = joblib.load(PRETRAINED_DIR / "scaler.pkl")
gesture_classes = np.load(PRETRAINED_DIR / "gesture_classes.npy", allow_pickle=True)

# Note: imu_dim_final and tof_thm_aggregated_dim_final are not needed for inference here
# as the predict function works on the full feature set.

custom_objs = {
    'time_sum': time_sum, 'squeeze_last_axis': squeeze_last_axis, 'expand_last_axis': expand_last_axis,
    'se_block': se_block, 'residual_se_cnn_block': residual_se_cnn_block, 'attention_layer': attention_layer,
}

models = []
print(f"  Loading {N_SPLITS} models for ensemble inference...")
for fold in range(N_SPLITS):
    # --- [重要] 推論時にはカスタムオブジェクトに`build_gated_two_branch_model`は不要 ---
    # 保存されたモデルはレイヤーの構造を保持しているため、カスタムレイヤー/関数のみ渡せばよい
    model_path = PRETRAINED_DIR / f"gesture_model_fold_{fold}.h5"
    model = load_model(model_path, compile=False, custom_objects=custom_objs)
    models.append(model)
print("  Models, scaler, feature_cols, pad_len loaded – ready for evaluation")

In [ ]:
# --- [新機能] TTA用のハイパーパラメータ ---
TTA_STEPS = 10  # TTAの実行回数。5〜10あたりが一般的。
TTA_NOISE_STDDEV = 0.01 # 入力データに加えるノイズの標準偏差

# --- [変更点] TTAを実装したpredict関数 ---
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    df_seq = sequence.to_pandas()
    
    # --- 1. 特徴量エンジニアリング（変更なし） ---
    linear_accel = remove_gravity_from_acc(df_seq, df_seq)
    df_seq['linear_acc_x'], df_seq['linear_acc_y'], df_seq['linear_acc_z'] = linear_accel[:, 0], linear_accel[:, 1], linear_accel[:, 2]
    df_seq['linear_acc_mag'] = np.sqrt(df_seq['linear_acc_x']**2 + df_seq['linear_acc_y']**2 + df_seq['linear_acc_z']**2)
    df_seq['linear_acc_mag_jerk'] = df_seq['linear_acc_mag'].diff().fillna(0)
    angular_vel = calculate_angular_velocity_from_quat(df_seq)
    df_seq['angular_vel_x'], df_seq['angular_vel_y'], df_seq['angular_vel_z'] = angular_vel[:, 0], angular_vel[:, 1], angular_vel[:, 2]
    df_seq['angular_distance'] = calculate_angular_distance(df_seq)
    for i in range(1, 6):
        pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]; tof_data = df_seq[pixel_cols].replace(-1, np.nan)
        df_seq[f'tof_{i}_mean'], df_seq[f'tof_{i}_std'], df_seq[f'tof_{i}_min'], df_seq[f'tof_{i}_max'] = tof_data.mean(axis=1), tof_data.std(axis=1), tof_data.min(axis=1), tof_data.max(axis=1)
        
    mat_unscaled = df_seq[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32')
    mat_scaled = scaler.transform(mat_unscaled)
    pad_input = pad_sequences([mat_scaled], maxlen=pad_len, padding='post', truncating='post', dtype='float32')

    # --- 2. TTAループの実行 ---
    all_tta_predictions = []
    for _ in range(TTA_STEPS):
        # 元の入力データにノイズを加える (TTA_STEPS=1回目はノイズなしでも良い)
        if TTA_STEPS > 1 and _ > 0: # 最初の1回は元のデータで予測
             noisy_input = pad_input + tf.random.normal(shape=tf.shape(pad_input), mean=0.0, stddev=TTA_NOISE_STDDEV)
        else:
             noisy_input = pad_input

        # 5フォールドモデルでアンサンブル予測
        all_fold_predictions = []
        for model in models:
            # 主出力(ジェスチャー確率)のみを取得
            main_preds, _ = model.predict(noisy_input, verbose=0)
            all_fold_predictions.append(main_preds)
        
        # フォールド間の予測を平均
        avg_fold_prediction = np.mean(all_fold_predictions, axis=0)
        all_tta_predictions.append(avg_fold_prediction)

    # --- 3. TTAの結果を最終的に平均化 ---
    final_avg_prediction = np.mean(all_tta_predictions, axis=0)
    
    # 最も確率の高いクラスのインデックスを取得
    idx = int(final_avg_prediction.argmax())
    
    return str(gesture_classes[idx])

In [ ]:
if not TRAIN:
    import kaggle_evaluation.cmi_inference_server
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(
            data_paths=(
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )